In [62]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import pickle
import matplotlib.pyplot as plt

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


# Pose Graph Optimization

### GOAL: Adjust configuration of absolute poses (nodes) to minimize squared error intorduced by constraints (edges)


Node: [x, y, z, r, p, y]

Edge: Odometry Measurements

# State vector $x$

absolute position and orientation of vehicle at each timestep

\begin{equation}
\textbf{x} = 
\begin{bmatrix}
\textbf{x}_1\\
\textbf{x}_2\\
\textbf{x}_3 \\
\vdots
\end{bmatrix} = 
\begin{bmatrix}
x_1 & y_1 & z_1 & r_1 & p_1 & y_1 \\
x_2 & y_2 & z_2 & r_2 & p_2 & y_2 \\
x_3 & y_3 & z_3 & r_3 & p_3 & y_3 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \\ 
\end{bmatrix}
\end{equation}

# Transformations $\mathbf{X}$  
Represeneted in Homogenous Coordinates. This is an overparameterization (similar to using quaternions) which helps avoid singularities when calculating the error function to adjust each Gauss-Newton iteration, however, the additional degrees of freedom need to be removed before forming our state vector

\begin{equation}
\mathbf{X_i} = 
\begin{bmatrix}
R_{11} & R_{12} & R_{13} & dx\\
R_{21} & R_{22} & R_{23} & dy \\
R_{31} & R_{32} & R_{33} & dz \\
0 & 0 & 0 & 1
\end{bmatrix} 
\end{equation}


$\big(\mathbf{X}_i^{-1} \mathbf{X}_{i+1} \big)$  describes how node $i$ sees node $(i+1)$ (ex: odometry)

$\big(\mathbf{X}_i^{-1} \mathbf{X}_{j} \big)$  describes how node $i$ sees node $j$ (ex: loop closure)

In [2]:
def v2t(vector):
    """converts a transformation vector to homogenous coordinate system"""
    if len(tf.shape(vector)) == 1: #allow for 1-D or N-D input
        vector = vector[None,:]
    angs = vector[:,3:]
    phi = angs[:,0]
    theta = angs[:,1]
    psi = angs[:,2]
    rot = tf.Variable([[cos(theta)*cos(psi), sin(psi)*cos(phi) + sin(phi)*sin(theta)*cos(psi), sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi)],
                       [-sin(psi)*cos(theta), cos(phi)*cos(psi) - sin(phi)*sin(theta)*sin(psi), sin(phi)*cos(psi) + sin(theta)*sin(psi)*cos(phi)],
                       [sin(theta), -sin(phi)*cos(theta), cos(phi)*cos(theta)]])
    rot = tf.transpose(rot, [2, 0, 1])
    trans = vector[:,:3]
    trans = np.reshape(trans, (np.shape(rot)[0], 3, 1))
    transform = tf.concat((rot, trans), axis = -1)
    extra = tf.tile(tf.constant([[[0., 0., 0., 1.]]], dtype = tf.double), (np.shape(rot)[0],1,1))
    transform = tf.concat((transform, extra), axis = -2)
    return transform

In [3]:
def t2v(mat):
    """converts transformation matrix to state vector"""
    if len( tf.shape(mat) ) == 2:
        mat = mat[None, :, :]
    R_sum = np.sqrt(( mat[:,0,0]**2 + mat[:,0,1]**2 + mat[:,1,2]**2 + mat[:,2,2]**2 ) / 2)
    phi = np.arctan2(-mat[:,1,2],mat[:,2,2])
    theta = np.arctan2(mat[:,0,2], R_sum)
    psi = np.arctan2(-mat[:,0,1], mat[:,0,0])
    angs = np.array([phi, theta, psi])
    vector = tf.concat((mat[:,:3,-1], angs.T), axis =1)
    return vector

In [4]:
#NOTE: v2t() -> t2v() is returning to the same value only if I take inverse (for rotation) 
#                     and without inverse only (for translation). [T]^-1 @ [T] = I still
#Question: is this normal?  

# # test = np.array([1., 2., 3., 0.05, 0.00, -0.14])
# test = np.ones([2,6])
# test[1,:] = np.array([1., 2., 3., 0.003, 0.001, 0.3])
# T = v2t(test)
# # print("T: \n",T)
# vect_trans = t2v(T).numpy()[:,:3]
# vect_rot = t2v(tf.linalg.inv(T)).numpy()[:,3:]
# print(np.append(vect_trans, vect_rot, axis = 1))

# Least Squares Error Function


The optimial state vector, $x^*$, occurs where:

\begin{equation}
\Large
x^* = \arg\min_x \sum_{ij}^{} e^T_{ij}(x_i,x_j)\Omega_{ij}e_{ij}(x_i,x_j)
\end{equation}

$\Omega_{ij}$ is the information matrix associated with the odometry estimate that relates $i$ and $j$. $\Omega_{ij}$ is the inverse of the covariance matrix $\sigma_{ij}$

the error funcion for each connected node $i,j$ as a function of the state vector $x$ is defined as:

\begin{equation}
\Large
e_{ij}(x) = \text{t2v}(Z_{ij}^{-1} (X_j^{-1}X_i))
\end{equation}

\begin{equation}
Z_{ij}^{-1} = \text{constraint (from measurement)}
\end{equation}

\begin{equation}
(X_j^{-1}X_i) = x_i \text{ relative to }x_j \text{ given the current model of system} 
\end{equation}

$ \text{Stachniss and Grisetti use} \boxplus \text{to repersent the mapping of euclidian space } v \text{ to a homogenous coordinate frame } t$


Important Note: $e_{ij}(x)$ only depends on $x_i$ and $x_j$ 

In [209]:
def get_e_ij(Zij, Xij):
    """calculates error function w.r.t. nodes i and j
    Zij == pose j relative to i according to nodes (rotation matrix)
    Xij == pose j relative to i according to constraints (rotation matrix)
    """        
    e_ij = t2v(tf.linalg.pinv(Zij) @ Xij)

    return(e_ij)    

In [220]:
Zij = v2t(np.array([1.,  1.9, 0., 0.0003, 0.0005, 0.01 ])) # get from graph 
Xij = v2t(np.array([1.1, 2.0, 0., 0.,     0.0,    0.008])) # get from odometry message
# Zij =  v2t(np.zeros([2,6]))
# Xij =  v2t(np.random.randn(2,6))

e_ij = get_e_ij(Zij, Xij)
# print(e_ij)

# Linearizing the System

\begin{equation}
\Large
e_{ij}(x + \Delta x) \approx e_{ij}(x) + J_{ij} \Delta x
\end{equation}

Here, $J_{ij}$ is the the jacobian of $e_{ij}$ with respect to x

\begin{equation}
\Large
J_{ij} = \frac{\delta e_{ij}(x)}{\delta x} = \bigg{(} 0 \dots \frac{\delta e_{ij}(x_i)}{\delta x_i}
\dots \frac{\delta e_{ij}(x_j)}{\delta x_j} \dots 0 \bigg{)} = \bigg{(} 0 \dots A_{ij} \dots B_{ij} \dots 0 \bigg{)} 
\end{equation}

\begin{equation}
A_{ij}, B_{ij} \in \mathbb{R}^{6 \times 6}
\end{equation}

\begin{equation}
J_{ij}(x) \in \mathbb{R}^{6 \times 6N}
\end{equation}
where $N$ is the total number of poses being solved for (nodes)

In [221]:
def get_A_ij_B_ij(e_ij):
    """calculates nonzero terms from the Jacobian of error function w.r.t. nodes i and j using TensorFlow
        e_ij == error function [x, y, z, phi, theta, psi]
        
        NOTE: this works with batch operations: error vectors passed in as tensor will result in
                corresponding output in the same order 
        
    """
    e_ij = tf.cast(e_ij, tf.float32)
    
    p_point = e_ij[:,:3]
    phi = e_ij[:,3][:,None]
    theta = e_ij[:,4][:,None]
    psi = e_ij[:,5][:,None]
    
    eyes = tf.tile(-tf.eye(3)[None,:,:], [tf.shape(p_point)[0] , 1, 1])
    
#     deriv of R() wrt phi
    dRdPhi = tf.Variable([[tf.zeros(len(phi), dtype = phi.dtype)[:,None], (-sin(psi)*sin(phi) + cos(phi)*sin(theta)*cos(psi)), (cos(phi)*sin(psi) + sin(theta)*sin(phi)*cos(psi))],
                       [tf.zeros(len(phi), dtype = phi.dtype)[:,None], (-sin(phi)*cos(psi) - cos(phi)*sin(theta)*sin(psi)), (cos(phi)*cos(psi) - sin(theta)*sin(psi)*sin(phi))], 
                       [tf.zeros(len(phi), dtype = phi.dtype)[:,None], (-cos(phi)*cos(theta)), (-sin(phi)*cos(theta))] ])[:,:,:,0]
    dRdPhi = tf.transpose(dRdPhi, (2,0,1))
    Jx = dRdPhi @ p_point[:,:,None]
    
    # (deriv of R() wrt theta).dot(p_point)
    dRdTheta = tf.Variable([[(-sin(theta)*cos(psi)), (cos(theta)*sin(phi)*cos(psi)), (-cos(theta)*cos(phi)*cos(psi))],
                               [(sin(psi)*sin(theta)), (-cos(theta)*sin(phi)*sin(psi)), (cos(theta)*sin(psi)*cos(phi))],
                               [(cos(theta)), (sin(phi)*sin(theta)), (-sin(theta)*cos(phi))] ])[:,:,:,0]
    dRdTheta = tf.transpose(dRdTheta, (2,0,1))
    Jy = dRdTheta @ p_point[:,:,None]

    # deriv of R() wrt psi
    dRdPsi = tf.Variable([[(-cos(theta)*sin(psi)), (cos(psi)*cos(phi) - sin(phi)*sin(theta)*sin(psi)), (cos(psi)*sin(phi) + sin(theta)*cos(phi)*sin(psi)) ],
                                       [(-cos(psi)*cos(theta)), (-sin(psi)*cos(phi) - sin(phi)*sin(theta)*cos(psi)), (-sin(phi)*sin(psi) + sin(theta)*cos(psi)*cos(phi))],
                                       [tf.zeros(len(phi), dtype = phi.dtype)[:,None],tf.zeros(len(phi), dtype = phi.dtype)[:,None],tf.zeros(len(phi), dtype = phi.dtype)[:,None]]])[:,:,:,0]
    dRdPsi = tf.transpose(dRdPsi, (2,0,1))
    Jz = dRdPsi @ p_point[:,:,None]
    
    top = tf.concat([eyes, Jx, Jy, Jz], axis = 2)
    flipped = tf.concat([tf.transpose(Jx, (0,2,1)), 
                         tf.transpose(Jy, (0,2,1)), 
                         tf.transpose(Jz, (0,2,1))], axis = 1)
    bottom = tf.concat([flipped, eyes], axis = 2)
    A_ij = tf.concat([top, bottom], axis = 1)
    B_ij = -A_ij
    
    return A_ij, B_ij

In [222]:
A_ij, B_ij = get_A_ij_B_ij(e_ij)
# print("A_ij:\n", A_ij.numpy().astype(np.float16), "\n B_ij: \n", B_ij.numpy().astype(np.float16))


# Accumumlate Sparse Tensors

calculate jacobians in batch operation first, then bring to sparse tensor for matmul??

In [199]:
st_a = tf.sparse.SparseTensor(indices=[[0, 2], [3, 4]],
                       values=[31.0, 2.0], 
                       dense_shape=[4, 6])
st_b = tf.sparse.SparseTensor(indices=[[0, 2], [3, 0]],
                       values=[56.0, 38.0],
                       dense_shape=[4, 6])
st_sum = tf.sparse.add(st_a, st_b)
# print(st_sum)
# print(tf.linalg.pinv(tf.sparse.to_dense(st_sum)).numpy())

Similar to ICET, we can use Newton-Raphson to iteratively solve for small perterbations to the state vector $x$ that bring us towards a better solution

\begin{equation}
\Large
x \rightarrow x + \Delta x
\end{equation}

\begin{equation}
\Large
H \Delta x = -b
\end{equation}

\begin{equation}
\Large
b^T = \sum_{ij}^{}b_{ij} = \sum_{ij}^{}e_{ij}^T \Omega_{ij} J_{ij}
\end{equation}

\begin{equation}
\Large
H = \sum_{ij}^{}H_{ij} = \sum_{ij}^{}J_{ij}^T \Omega J_{ij}
\end{equation}

Init Hessian once at start of simulation, update in place as new linearizations are added(?)

In [306]:
# TRY: tf.linalg.cholesky_solve
indices = tf.constant([[1,3], [3,1], [5,5]], dtype = tf.int64) #[N, ndims]
values = tf.constant([10.5, 10.5, -0.55])
dense_shape = tf.constant([6, 6], dtype = tf.int64)
sparse_J =  tf.sparse.SparseTensor(indices, values, dense_shape)
print(sparse_J)
print(tf.sparse.to_dense(sparse_J))
print(tf.linalg.det(tf.sparse.to_dense(sparse_J)))

SparseTensor(indices=tf.Tensor(
[[1 3]
 [3 1]
 [5 5]], shape=(3, 2), dtype=int64), values=tf.Tensor([10.5  10.5  -0.55], shape=(3,), dtype=float32), dense_shape=tf.Tensor([6 6], shape=(2,), dtype=int64))
tf.Tensor(
[[ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.   10.5   0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.   10.5   0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.55]], shape=(6, 6), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)


In [308]:
# test = tf.linalg.cholesky(tf.sparse.to_dense(sparse_J)[None,:,:] + tf.eye(10)[None,:,:]) #needs to be symmetric and positive definite
test = tf.sparse.to_dense(sparse_J)[None,:,:] + 10*tf.eye(6)[None,:,:]
print(tf.linalg.eig(test)[0])
print(test)

tf.Tensor(
[[10.        +0.j 20.5       +0.j -0.49999976+0.j 10.        +0.j
  10.        +0.j  9.45      +0.j]], shape=(1, 6), dtype=complex64)
tf.Tensor(
[[[10.    0.    0.    0.    0.    0.  ]
  [ 0.   10.    0.   10.5   0.    0.  ]
  [ 0.    0.   10.    0.    0.    0.  ]
  [ 0.   10.5   0.   10.    0.    0.  ]
  [ 0.    0.    0.    0.   10.    0.  ]
  [ 0.    0.    0.    0.    0.    9.45]]], shape=(1, 6, 6), dtype=float32)


# Apply Constraint to First Node in Kinematic Chain

\begin{equation}
\Large
H_{11} \rightarrow H_{11} + I
\end{equation}

Optimization routine can't make sense of only relative measurements. We need to specify the first pose as the origin

In [235]:
# Load test data
odometry_history = np.load("test_data/leddartech_pixset/T_vec_history.npy")
cov_estimate_history = np.load("test_data/leddartech_pixset/cov_vec_history.npy")

In [233]:
def get_information_matrix(cov_estimate):
    """returns information matrix sigma from ICET cov_estimates"""
    
    print(np.shape(cov_estimate_history))
    
    #convert predicted stds -> covariance matrix
    
    #invert
    
    #convert to TF Tensor
    
    sigma = None #temp
    return sigma
    
sigma = get_information_matrix(cov_estimate_history[:,:6])

(185, 8)


# TODO

Figure out references in Jupyter Notebook


# Questions

Should $A_{ij} = -B_{ij}$?

Are there advantages to reporting state vector in unit quaternions? Besides being less intuitive to work with I've read they can also introduce problems in optimization routine since they add an additional degree of freedom. Every guide I've seen so far uses homogenous coordinate representation for the transformation matrices when computing the loss function, since they can be applied neatly in series. So rotation matrices do not produce singularities?

Is what we did in ICET to handle sparsity (i.e. accumulating contributions for corresponding elements rather than performing full matrix inversion) similar to a Choelesky Decomposition?

<span style="color:red"> What is going on with the numerics in get_e_ij()? Why is it not just t2v(Zij) - t2v(Zij)?? </span>



# Potential Contributions

Use ICET error covaraince estimation to demonstrate improvement in accuracy for Volpe Dataset

Use ICET output to inform $\Omega$ as a spatially dependant weighting field rather than simply indexing each unique point. Explore Gaussian Process Regression?


In [249]:
test = tf.random.normal([600,600])
# print(tf.linalg.pinv(test))

https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.inv.html#scipy.sparse.linalg.inv